In [10]:
import numpy as np
from define import qcode as qc
from define import globalvars as gv
from define.randchans import RandomUnitary
from define.QECCLfid.utils import Dot, Dagger

In [2]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [3]:
# Pr(synd) = sum_s,s' GS_ss' P(synd)_s'

In [4]:
def LoadChannel():
    return np.loadtxt("./../input/debug_testing/physical.txt").reshape(256, 256)
def LoadProjector():
    return np.loadtxt("./../input/debug_testing/SS.txt").reshape(64, 64)

In [5]:
def SyndromeTest(GS, P):
    # Test if any syndrome probabilities are negative.
    norm = 0
    stop = 0
    for synd in range(64):
        Pr_s = 0
        if stop == 0:
            print("P[{}, :] has {} 1s and {} -1s.".format(synd, np.count_nonzero(P[synd, :] == 1), np.count_nonzero(P[synd, :] == -1)))
        Pr_s = np.sum(np.dot(GS, P[synd, :]))/64
        norm = norm + Pr_s
        if stop == 0:
            print("Pr({}) = {}".format(synd, Pr_s))
        if Pr_s < 0:
            stop = 1
            print("Negative Pr(s) encountered.")
    print("Sum of all syndrome probabilities: {}.".format(norm))
    return None

In [6]:
# We want to construct the S x S part of the process matrix for a random independent unitary channel.
# G_ij = prod_q Tr[ U_q  (P_i)_q U^dag_q  (P_j)_q ]

In [16]:
def GetRandUnitaryProcess(nq):
    U = np.zeros((nq, 2, 2), dtype=np.complex128)
    for q in range(nq):
        U[q, :, :] = RandomUnitary(prox=0.1, dim=2)
    S = qc.GenerateGroup(qcode.S)
    GS = np.zeros((S.shape[0], S.shape[0]), dtype = np.double)
    for i in range(S.shape[0]):
        for j in range(S.shape[0]):
            contrib = 1 + 0 * 1j
            for q in range(nq):
                contrib = contrib * np.trace(Dot(U[q, :, :], gv.Pauli[S[i, q], :, :], Dagger(U[q, :, :]), gv.Pauli[S[j, q], :, :]))
            GS[i, j] = np.real(contrib)/np.power(2, nq)
    return GS

In [8]:
GS = LoadChannel()[:64, :64]
P = LoadProjector()
SyndromeTest(GS, P)

P[0, :] has 64 1s and 0 -1s.
Pr(0) = 0.017761668739342124
P[1, :] has 32 1s and 32 -1s.
Pr(1) = 0.029819225617637606
P[2, :] has 32 1s and 32 -1s.
Pr(2) = 0.01905007000606568
P[3, :] has 32 1s and 32 -1s.
Pr(3) = 0.01428886061133896
P[4, :] has 32 1s and 32 -1s.
Pr(4) = -0.00011155629058838372
Negative Pr(s) encountered.
Sum of all syndrome probabilities: 0.9999999999999996.


In [21]:
GS = GetRandUnitaryProcess(7)
P = LoadProjector()
SyndromeTest(GS, P)

P[0, :] has 64 1s and 0 -1s.
Pr(0) = 0.8948582415687351
P[1, :] has 32 1s and 32 -1s.
Pr(1) = 0.010721054150963222
P[2, :] has 32 1s and 32 -1s.
Pr(2) = 0.03070671339438824
P[3, :] has 32 1s and 32 -1s.
Pr(3) = 0.004082922464688388
P[4, :] has 32 1s and 32 -1s.
Pr(4) = 0.002512122528410432
P[5, :] has 32 1s and 32 -1s.
Pr(5) = 0.006774318488679182
P[6, :] has 32 1s and 32 -1s.
Pr(6) = 0.00023409731732307315
P[7, :] has 32 1s and 32 -1s.
Pr(7) = 0.003305722312695436
P[8, :] has 32 1s and 32 -1s.
Pr(8) = 1.70209925341204e-06
P[9, :] has 32 1s and 32 -1s.
Pr(9) = 0.0018141528053659782
P[10, :] has 32 1s and 32 -1s.
Pr(10) = 7.815124954126465e-06
P[11, :] has 32 1s and 32 -1s.
Pr(11) = 6.195144575134315e-05
P[12, :] has 32 1s and 32 -1s.
Pr(12) = 1.5368998176727017e-05
P[13, :] has 32 1s and 32 -1s.
Pr(13) = 4.783811773769256e-06
P[14, :] has 32 1s and 32 -1s.
Pr(14) = 6.264355250921311e-06
P[15, :] has 32 1s and 32 -1s.
Pr(15) = 5.19271198531554e-07
P[16, :] has 32 1s and 32 -1s.
Pr(16) =